# An intermediate user's guide to RegEx

In [3]:
import re

### Prerequisites
The target audience for this article are people who:
- Understand basic Python syntax (strings, lists, loops)
- Understand what an object is, what a function is
- Understand the very basics of regular expressions, in that they are used for string processing tasks and some simple metacharacters like `*`, `?`, `\d`
- Want to expand their RegEx toolkit beyond the basic patterns

### Context
My main motivation for further researching the `re` module (Python's regex module), and writing about it, was that I have often relied on using RegEx in my code, but outside of very simple expressions, I was never comfortable using it in enterprise level code (code that is shared by my team and deployed for our customers). I have relied on Stack Overflow for very simple regular expressions but nothing beyond because I didn't fully understand the more complex ones.

And despite how well written some of the Stackoverflow posts are, there were many doubts in my mind. For example I vividly remember seeing `(?:...)` in some answers, but not appreciating why the author put `?:` inside the parentheses. I tried doing an AB test (with and without the `?:`), and I couldn't deciper the effect this syntax has. Now that I have read much more about RegEx and tried it out, I know this construct is called a 'non-capturing group'. I will unpack this and other features of `re` in this article, which, I think, if you understand, will make you an intermediate user of RegEx.

To give you another spoiler (which will create more context for this article), the core building block on which intermediate usage of regex is built, is Grouping. And that's the core learning I'm trying to share in this article. Many powerful regex features, mentioned here and otherwise, build on top of Grouping, so it will serve you well to understand Grouping well.

##### Side note:
The reason I think these are intermediate features, is because you get by without knowing these, and still implement simple regexes in real code.

At the same time, I don't think these are Advanced either. Those would include Sub-routines and really exploring look-arounds, as well as recursion (Yeah, I know. It's crazy that you can do recursion in Regex)

### 2 (intermediate level) primers before we discuss more RegEx

#### 1. Raw Strings
In Python, we usually create RegEx strings using 'raw strings'. A Python raw string is created by prefixing a string literal with 'r' or 'R', like so: `r"Hello world"`

The main difference between regular strings and raw strings is that raw strings treat the backslash(`\`) as a literal character. However, in regular Python strings, backslashes are used to indicate escape sequences like `\n`, `\r` or `\t`. In other words, the raw string `r"\n"` is two characters `\` and `n`, but the regular string `"\n"` is one character, namely `\n`.

So when we want to create a string that contains backslash(`\`) and don’t want it to be treated as an escape character, we use raw strings.

__Why is this relevant in regex__<br>
It's recommended that you use raw strings to create RegEx patterns, because the RegEx engine interprets backslashes in a special way. For example, when you put `\d` in a regex pattern, it indicates (to the RegEx engine) a digit between 0 to 9. But the Python interpreter interprets backslashes in strings in a different, non-literal, way as well, which creates a conflict when using backslashes in regular strings in Python. In fact, `\d` is an invalid escape sequence according to the Python interpreter.

Moreover, if you use `\d` in a regular Python string, the Python interpreter will think you're trying to create an escape sequence and you'll get a Syntax Warning that `\d` is an invalid escape sequence, because it doesn't mean anything to Python. Plus, I believe that in future versions, you'll get a syntax error for the same, so it will make even more sense to use raw strings to create such RegEx strings.

If you read some of the Stack Overflow answers about RegEx, you will see that developers prefer to always use raw strings for RegEx, because with raw strings, what you see is what you get.

That being said, raw strings are not strictly necessary. You can 'escape' your backslashes in your regular strings by putting another backslash before them, but that becomes ugle very quickly. See here: https://docs.python.org/3/howto/regex.html#the-backslash-plague

#### 2. Module level functions vs. Object level functions
The `re` module allows you to call its matching functions in two (mostly) equivalent ways - either via a Pattern Object, or on the module level itself.

There are 4 main functions to match a regex in a string, which are available in both modes of calling
- `match()` - Returns first matching substring as a `re.Match` object, if found at the start of string
- `search()` - Returns first matching substring as a `re.Match` object, found anywhere in the string
- `findall()` - Returns all matching substrings as a list of strings
- `finditer()` - Returns an interater of `re.Match` objects, for all matching substrings in the string

Let's discuss the first way, which is by invoking these functions via a __Pattern object__. A Pattern object is what you get when you compile a regex string (`re.compile`). And the above functions become available as attributes of the pattern object, which can be called in the following way:

In [31]:
# Maybe break up the following cell into smaller cells before putting it in article
regex_string = r'(?:1st|2nd|3rd|\dth) article'
p = re.compile(regex_string)

print(f"p is a {type(p)} object\n")

test_string = "My 3rd article and 2nd article"

print(f"p.match(test_string) = {p.match(test_string)}\n")

print(f"p.search(test_string) = {p.search(test_string)}\n")

print(f"p.findall(test_string) = {p.findall(test_string)}\n")

matches_iterable = p.finditer(test_string)
print(f"p.finditer(test_string) = {matches_iterable}")
for i, match in enumerate(matches_iterable):
    print(f"Match # {i} = {match}")
    
    
# TODO: In the article, consider using screenshots of doing this in a Terminal

p is a <class 're.Pattern'> object

p.match(test_string) = None

p.search(test_string) = <re.Match object; span=(3, 14), match='3rd article'>

p.findall(test_string) = ['3rd article', '2nd article']

p.finditer(test_string) = <callable_iterator object at 0x7fbbbe1efbe0>
Match # 0 = <re.Match object; span=(3, 14), match='3rd article'>
Match # 1 = <re.Match object; span=(19, 30), match='2nd article'>


The second way is to simply invoke these functions on the 'Module level', but you have to specify the RegEx string each time, like so:

In [68]:
# Maybe break up the following cell into smaller cells before putting it in article
regex_string = r'(?:1st|2nd|3rd|\dth) article'
print(f"regex_string = {regex_string}\nBut there's no need to compile it\n")
test_string = "My 3rd article and 2nd article"

print(f"re.match(regex_string, test_string) = {re.match(regex_string, test_string)}\n")

print(f"re.search(regex_string, test_string) = {re.search(regex_string, test_string)}\n")

print(f"re.findall(regex_string, test_string) = {re.findall(regex_string, test_string)}\n")

matches_iterable = re.finditer(regex_string, test_string)
print(f"re.finditer(regex_string, test_string) = {matches_iterable}")
for i, match in enumerate(matches_iterable):
    print(f"Match # {i} = {match}")

regex_string = (?:1st|2nd|3rd|\dth) article
But there's no need to compile it

re.match(regex_string, test_string) = None

re.search(regex_string, test_string) = <re.Match object; span=(3, 14), match='3rd article'>

re.findall(regex_string, test_string) = ['3rd article', '2nd article']

re.finditer(regex_string, test_string) = <callable_iterator object at 0x0000029BBBC92550>
Match # 0 = <re.Match object; span=(3, 14), match='3rd article'>
Match # 1 = <re.Match object; span=(19, 30), match='2nd article'>


<b><u>Takeaway</u></b>:There's 2 convenient ways to use regex matching methods. Both will produce the same results. The only reason you may favor the __Pattern level__ functions is to save the compiled pattern once and use it again. As opposed to __Module level__ functions that re-compile the regex pattern every time you want to match it with a string.

This is somewhat of a time save, and arguably makes your code cleaner. In fact, the Module level functions automatically store compiled objects in a cache, so realistically, there isn't a big difference unless you have a lot of regex calls using different regex patterns.
    
The reason I bring it up here is because you may see either of these style in other people's code or Stack Overflow posts, so you should remember that they do the same thing.

### Grouping

Finally, let's dive into Groups.

In RegEx strings, Groups are marked by the `(` and `)` metacharacters. Conceptually, they are interpreted in the same way parentheses are treated in mathematical expressions.
<br><br>For example, if you see `(2+3)*5` in algebra, you know that the `2+3` is its own entity, or a group, and by the rule of BODMAS, it's evaluated before anything else.
<br><br>Similarly, `r"(ab)c"` in RegEx means `ab` is a group, and you can use grouping with other metacharacters to make powerful RegExes. For example `r"(ab)*c"` will catch expressions where you can have 0 or more instances of `ab` followed by a `c`.

In [3]:
regex_with_group = r'(ab)*c'
print(re.search(regex_with_group, 'c'))
print(re.search(regex_with_group, 'abc'))
print(re.search(regex_with_group, 'ababc'))

<re.Match object; span=(0, 1), match='c'>
<re.Match object; span=(0, 3), match='abc'>
<re.Match object; span=(0, 5), match='ababc'>


#### Grouping feature #1
Groups can actually be separately/ individually retrieved using `.group()` or `.groups()` on the Match Object

In [4]:
regex_with_group = r'(ab)*c'
m = re.search(regex_with_group, 'ababc') # m is the Match Object
m.group(1) # Group indexes start at 1

'ab'

Here we retrieved the first group. Yes, indexing of groups starts at 1.
<br><br>Actually `m.group(0)` will give you the whole matched string (in the above case that's `abc`). And this is equivalent to passing no argument `m.group()`, since the default argument value is 0.
<br><br>In fact, when you see Stackoverflow regex posts, most of them return the matched string by calling `m.group()` (in other words, calling `.group()` on the match object `m`). Again, this is the same as `m.group(0)` because 0 is the default argument of `.group([group1,...])`.

#### Grouping feature #2
You can even reference a group later in the same regex using a backslash and the index of the group

In [16]:
regex_with_group_backref = r'(ab)*c\1'
m = re.search(regex_with_group_backref, 'ababcabab')
print(m.group(0))

ababcab


Here, the `\1` matches with the group `ab` since that is the first group in the regex

#### Grouping feature #3
<b>Non-capturing groups

When your group starts with a question mark and colon `?:`, it's a non-capturing group. So for example, `r"(?:ab)c"` is the same as before, but `ab` as a group is 'not captured'. Meaning it can't be identified using functions like `m.group(1)` or referenced later in the regex using backslashes like `\1`. Let's see an example:

In [28]:
regex_with_nc_group_backref = r'(?:ab)*c\1'
try:
    m = re.search(regex_with_nc_group_backref, 'ababcabab')
    print(m.group(0))
except Exception as e:
    print("Exception:", e)

Exception: invalid group reference 1 at position 9


Here we saw, the program ran into an Exception. We tried referencing a group later in the expression but there was no _captured_ group to reference.

In [29]:
regex_with_nc_group = r'(?:ab)*c'
try:
    m = re.search(regex_with_nc_group, 'ababc')
    m.group(1)
except Exception as e:
    print("Exception:", e)

Exception: no such group


Again, the program ran into an Exception. This time, we tried retrieving the first group (index starting at 1) from the matched object, but same as before, there was no _captured_ group.

You can also see that the non-capturing group won't show up when you call `.groups()` on the match object `m`, but a capturing group will. I'll get into `.groups()` in the next section, although it's exactly what you expect :)

That's it. This is the core Grouping logic. I encourage you to write some regular expressions and test them out on your own. That's how you will really internalize it.

The following features will rely on you to understand Grouping, but they themselves are very easy to understand.

### Intermediate RegEx feature #1
`m.group()` vs. `m.groups()`

One point of confusion for you may be when you see `m.group()` vs. `m.groups()` in code involving regular expressions. Now that you understand what Groups are in regular expressions, the difference between these two functions will be pretty straightforward to understand:
1. `m.groups()` will give you a list of all captured groups in the regular expression. As you can imagine, non-capturing groups will not be included. Docs: https://docs.python.org/3/library/re.html#re.Match.groups
2. `m.group([group1,...])` will give you the group specified by the index. You can even specify multiple groups, in which case it returns a tuple of those groups. And as explained before, if you say `m.group()` or `m.group(0)`, it will return the whole matched string. Docs: https://docs.python.org/3/library/re.html#re.Match.group

That's it. Let's see it in action

In [7]:
regex_with_groups = r"(ab)+c(de)\1f"

First, let's break down what's happening in this RegEx:
- There should be at least 1 `ab` at the start, but there can be as many of them, back to back. Also, this is our first group.
- This should be followed by `c`.
- This should be followed by `de`, and this is also our second group.
- This should be followed by our first group, which we know is `ab`.
- Finally, this should be followed by an `f`.

In [8]:
test_string = "blah blah ababcdeabf blah blah"
m = re.search(regex_with_groups, test_string)
if not m:
    print("Weird, no match found")
else:
    print("All groups:", m.groups())
    print("Group 0:", m.group(0)) # Same as m.group()
    print("Group 1:", m.group(1))
    print("Group 2:", m.group(2))
    print("Group 0 and Group 1 together:", m.group(0,1))

All groups: ('ab', 'de')
Group 0: ababcdeabf
Group 1: ab
Group 2: de
Group 0 and Group 1 together: ('ababcdeabf', 'ab')


That's it, now you understand all there is to about `.group()` and `.groups()`

### Intermediate RegEx feature #2
`findall()` vs. `finditer()`

There is actually a nuance to `findall()`, which stumped me for a long time. I'll explain it in my words below, but for reference, it's summarized in the Python docs here - https://docs.python.org/3/library/re.html#re.findall.

`findall()`: As the name suggests, this returns a list of all matches in the order they are found. Here are the main subtleties:
- If you're not using any capturing groups in your RegEx, this will return a list of all strings that match with the whole RegEx pattern
- If you're using 1 capturing group in your RegEx, this will return a list of all strings that match that group (NOT the whole matched string)
- If you're using multiple capturing groups (more than 1) in your RegEx, this will return a list of tuples of all groups, for all matches, in the format `[..(<group 1>, <group 2>..., <group m>)..]`

See the below examples for clarity.

In [11]:
test_string = "blah blah ab collate for ab test but no ab c test? That's weird no? More like ab confusing"

In [12]:
regex_no_group = r"ab c"
re.findall(regex_no_group, test_string)

['ab c', 'ab c', 'ab c']

Pretty straightforward, just a list of all matches for the entire RegEx pattern `ab c` when no groups

In [13]:
regex_1_group = r"(ab) c"
re.findall(regex_1_group, test_string)

['ab', 'ab', 'ab']

This returns a list of all occurences of the first (and only) group in the pattern.

There's another subtlety here (that I didn't know before researching this article) - the substring `'ab test'` didn't get caught because it doesn't have `ab c` in it. Since in this case `.findall()` is only returning a list of all occurences of `ab`, you may think this will be returned, but it's not since the RegEx `r"(ab) c"` hasn't matched with that part of the string.

In [14]:
regex_more_groups = r"(ab) (c)"
re.findall(regex_more_groups, test_string)

[('ab', 'c'), ('ab', 'c'), ('ab', 'c')]

This returns a list of of all occurences of all groups, organized as a tuple of groups.

Of course, this is a dummy example so it seems useless. But look at the documentation linked above for a potential use case.

More importantly, I am including it here because I have tripped on this function a lot of times. The fact that it doesn't return the whole matched string when you use 1 or more capturing groups in the RegEx is crazy baffling to me, and has caused me a lot of pain :'( #feelsbadman.

It won't be surprising to you that I don't use `.findall()` in my workflows. If someone can point out a use for this weird function, that can't be solved by `.finditer()` (explained below), please let me know in the comments below.

`finditer()`: This returns an iterator (read Lazy Evaluation) to iterate over all matches of the entire RegEx pattern (not just any particular group). Docs: https://docs.python.org/3/library/re.html?highlight=regex#re.findall

Let's see it in action

In [57]:
re.finditer(regex_more_groups, test_string)

No surprises, it's an iterator object. And it hasn't been evaluated because we haven't asked for the results. Pretty clutch if you ask me.

In [59]:
for m in re.finditer(regex_more_groups, test_string):
    print("m is a", type(m))

m is a <class 're.Match'>
m is a <class 're.Match'>
m is a <class 're.Match'>


As I said, it's an iterator of `re.Match` objects

In [18]:
# Using the RegEx with multiple capturing groups
for m in re.finditer(regex_more_groups, test_string):
    print(m.group(0))

ab c
ab c
ab c


Let's use the RegEx with only 1 group

In [17]:
# Using the RegEx with 1 capturing group
for m in re.finditer(regex_1_group, test_string):
    print(m.group(0))

ab c
ab c
ab c


It's the same, no surprises here. `.finditer()` is the best :)

Plus, if I want to retrieve a particular group in a given match object, I can simply call `m.group(1)` (or whatever group index) since I have access to the match object in the iterator.

## Bonus Features

### Intermediate RegEx feature #3
Creating regex patterns using formatted strings

If you want to create regular expressions dynamically, read on. Sometimes, the pattern you want to catch in a string, and hence the pattern you want to encode in a regular expression, will depend on external things like user input, or the output of another piece of code.

This is a great use case for using formatted strings `f"..."`.

And Yes, you can create a __formatted raw string__! `fr"..."`. Let's see it in action

In [20]:
username = input("Please enter username: ") # Say the username is abcd123
regex_with_username = fr"{username}.*invalid" # This is the formatted raw string that is our RegEx pattern
text_with_username = "Something something... abcd123 is invalid"

re.search(regex_with_username, text_with_username).group()

Please enter username: abcd123


'abcd123 is invalid'

That's it. This is not rocket science, just a cool feature that you may not have known.

Actually, there's a small __nuance__ to formatted RegEx strings: You may want to use `{m, n}` in your regular expressions, a repetition qualifier, to specify exactly how many occurences you want to allow for a given sub expression. For example, `r"(ab){2,4}c"` means `ab` must occur at least 2 times and at most 4 times, and is then followed by a `c`.

But, `{...}` is used to define an evaluation in formatted strings. For example `f"{username} is invalid"` will result in the string `"abcd123 is invalid"`, assuming the variable `username` has been set to `abcd123`.

So, we need to 'escape' the curly braces, and this is accomplished by simply using double curly braces like so: `fr"(ab){{2,4}}c"`

In [40]:
username = input("Please enter username: ")
regex_with_username = fr"{username}.{{2,4}}invalid" # Changed this line
text_with_username = "Something something... abcd123 is invalid"
text_with_username_no_space = "Something something... abcd123 invalid"

print(re.search(regex_with_username, text_with_username))
print(re.search(regex_with_username, text_with_username_no_space))

Please enter username: abcd123
<re.Match object; span=(23, 41), match='abcd123 is invalid'>
None


Because the second text has fewer than 2 characters between `{username}` and `invalid`, it didn't match with our new RegEx (with the repetition qualifier).

There you go, you just learned another (semi) intermediate feature of RegEx ;)